In [1]:
import tensorflow as tf
from Fine_Tuning.GPT_2 import GPTModel
from Fine_Tuning.GPT_2 import Utils 
import torch
import tiktoken
import numpy as np
from torch.utils.data import Dataset
import pandas as pd

## GPT-2 124 Million Parameter

In [2]:
GPT_CONFIG_124M = {
    "vocab_size" : 50257,
    "context_length": 1024,
    "emb_dim" : 768,
    "n_heads" : 12,
    "drop_rate" : 0.1,
    "qkv_bias" : True,
    'n_layers' :12
}

tokenizer = tiktoken.get_encoding("gpt2")

device = "cpu"

In [3]:
gpt = GPTModel(GPT_CONFIG_124M)

In [5]:
gt = Utils()

## Loading Weights

In [4]:
checkpoint = torch.load("model_and_optimizer.pth");
gpt = GPTModel(GPT_CONFIG_124M);
gpt.load_state_dict(checkpoint['model_state_dict']);
optimizer = torch.optim.Adam(gpt.parameters(),lr=0.0004,weight_decay=0.1);
optimizer.load_state_dict(checkpoint['optimizer_state_dict']);
gpt.train();

C:\Users\Smile\AppData\Local\Temp\ipykernel_23972\1054121477.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("model_and_optimizer.pth");


In [6]:
def classify_review(text, model, tokenizer, device, max_length=None, pad_token_id=50256):
    model.eval()

    # Prepare inputs to the model
    input_ids = tokenizer.encode(text)
    supported_context_length = model.pos_emb.weight.shape[0]
    # Note: In the book, this was originally written as pos_emb.weight.shape[1] by mistake
    # It didn't break the code but would have caused unnecessary truncation (to 768 instead of 1024)

    # Truncate sequences if they too long
    input_ids = input_ids[:min(max_length, supported_context_length)]

    # Pad sequences to the longest sequence
    input_ids += [pad_token_id] * (max_length - len(input_ids))
    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0) # add batch dimension

    # Model inference
    with torch.no_grad():
        logits = model(input_tensor)[:, -1, :]  # Logits of the last output token
    predicted_label = torch.argmax(logits, dim=-1).item()

    # Return the classified result
    return "Positive" if predicted_label == 1 else "Negative"

In [12]:
text_1 = "Food was  Amazing and Pasta was good"


print(classify_review(
    text_1, gpt, tokenizer, device, max_length=40
))

Positive
